In [1]:
!pip uninstall mex -y

Found existing installation: mex 0.3
Uninstalling mex-0.3:
  Successfully uninstalled mex-0.3


In [2]:
!pip install --quiet "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

In [3]:
#np.set_printoptions(threshold=sys.maxsize)
import os
import sys
import pytz
import timeit
import datetime
import numpy as np
import scipy.io as sio

#scipy
from scipy.optimize import linprog
from pytest import approx

#mex
from mex.simplex import simplex_networks as mex_sn
from mex.simplex import problem_definition as mex_pd
from mex.utils import general as mex_g
from mex.simplex.minimizer_class import Minimizer
from mex.simplex.maximizer_class import Maximizer

## Características de la instancia

In [16]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              2
Core(s) per socket:              4
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
Stepping:                        7
CPU MHz:                         3104.861
BogoMIPS:                        4999.98
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       128 KiB
L1i cache:                       128 KiB
L2 cache:                        4 MiB
L3 cache:                        35.8 MiB
NUMA node0 CPU(s):               

In [164]:
mat = sio.loadmat('../data/DEGEN3.mat')   #Start time:  10:27:30 PM  End time:  10:30:58 PM
#mat = sio.loadmat('../data/SHIP04L.mat')  #Start time:  09:21:59 PM  End time:  09:21:59 PM
#mat = sio.loadmat('../data/AGG.mat')      #Start time:  09:52:51 PM  End time:  09:52:54 PM
#mat = sio.loadmat('../data/AGG2.mat')     #Start time:  09:53:47 PM  End time:  09:53:53 PM
#mat = sio.loadmat('../data/QAP8.mat')     #Start time:  10:11:35 PM  End time:  10:11:36 PM
#mat = sio.loadmat('../data/QAP15.mat')    #Start time:  10:15:30 PM  End time:  10:15:50 PM
#mat = sio.loadmat('../data/TRUSS.mat')    #Start time:  10:24:53 PM  End time:  10:24:54 PM

In [165]:
A = mat['A']
b = mat['b']
c = mat['c']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
A.shape

(1503, 2604)

# Prueba de paquetería `mex`

In [166]:
A = A.toarray()
b = np.array(b.T.tolist()[0])
c = np.array(c.T.tolist()[0])

minim = Minimizer(A,b,c)

In [167]:
minim.matrix.shape

(1504, 4109)

In [168]:
print("Start time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))
minim.solve()
print("End time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))

Start time:  10:27:30 PM
End time:  10:30:58 PM


**Valores aproximados**

In [169]:
min_approx = minim.get_min()

In [170]:
coeff_approx = minim.get_coeff()

In [171]:
coeff_approx

array([   0. ,    0. , 1409.1, ...,    0. ,    0. ,    0. ])

**Valores objetivo**

In [172]:
min_obj = linprog(c, A_ub=A, b_ub=b).fun

In [173]:
coeff_obj = linprog(c, A_ub=A, b_ub=b).x

In [174]:
coeff_obj

array([8.42195637e-02, 2.53827824e-09, 2.80069124e-09, ...,
       1.03070294e-09, 6.17118328e-02, 9.04065930e-10])

**Comprobación**

In [181]:
assert min_obj == approx(min_approx, abs=1e+5), "El valor aproximado es incorrecto"
#assert np.round(coeff_obj,3) == approx(coeff_approx, abs=1e+5), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", min_obj)
print("El valor aproximado obtenido con mex es: ", min_approx)
#print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj)
#print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx)

El valor objetivo obtenido con scipy es:  -1100.610006327479
El valor aproximado obtenido con mex es:  -1409.1000000000006
